# Fish Speech

### For Windows

In [ ]:
!chcp 65001

### For Linux

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

### Prepare Model

In [ ]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com 

!huggingface-cli download fishaudio/fish-speech-1.2-sft --local-dir checkpoints/fish-speech-1.2-sft/

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
!python tools/webui.py \
    --llama-checkpoint-path checkpoints/fish-speech-1.2-sft \
    --decoder-checkpoint-path checkpoints/fish-speech-1.2-sft/firefly-gan-vq-fsq-4x1024-42hz-generator.pth \
    # --compile

## Break-down CLI Inference

### 1. Encode reference audio

You should get a `fake.npy` by doing this.

In [ ]:
## Enter the path to the audio file here
src_audio = r"D:\PythonProject\\vo_hutao_draw_appear.wav"

!python tools/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "checkpoints/fish-speech-1.2-sft/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

### 2. Generate semantic tokens from text:
> This command will create codes_N files in the working directory, where N is an integer starting from 0.

> You can use --compile to fuse CUDA kernels for faster inference.

In [ ]:
!python tools/llama/generate.py \
    --text "hello world" \
    --prompt-text "The text corresponding to reference audio" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2-sft" \
    --num-samples 2
    # --compile

### 3. Generate speech from semantic tokens:

In [ ]:
!python tools/vqgan/inference.py \
    -i "codes_0.npy" \
    --checkpoint-path "checkpoints/fish-speech-1.2-sft/firefly-gan-vq-fsq-4x1024-42hz-generator.pth"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)